In [ ]:
# Funzione per caricare i dati
import matplotlib.pyplot as plt
import seaborn as sns
import os
from scipy.interpolate import make_interp_spline
import numpy as np

def load_data(resolution):
    train_df = pd.read_csv(f'src/train_scores_{resolution}.csv')
    test_df = pd.read_csv(f'src/test_scores_{resolution}.csv')
    metrics_df = pd.read_csv(f'src/metrics_scores_{resolution}.csv')
    
    # Aggiungi una colonna per la risoluzione
    train_df['resolution'] = resolution
    test_df['resolution'] = resolution
    metrics_df['resolution'] = resolution
    
    return train_df, test_df, metrics_df

# Carica i dati per tutte le risoluzioni
resolutions = [32, 64, 128]
train_dfs, test_dfs, metrics_dfs = [], [], []

for res in resolutions:
    train_df, test_df, metrics_df = load_data(res)
    train_dfs.append(train_df)
    test_dfs.append(test_df)
    metrics_dfs.append(metrics_df)

# Combina i dati in un singolo DataFrame per ogni tipo di dati
train_data = pd.concat(train_dfs)
test_data = pd.concat(test_dfs)
metrics_data = pd.concat(metrics_dfs)


def calculate_rolling_mean(series, window=50):
    # Calcola la media mobile
    return series.rolling(window=window, min_periods=1).mean()

def smooth_spline(x, y, num_points=50):  # Aumentato il numero di punti a 1000 per una maggiore levigatezza
    # Rimuove valori NaN per evitare errori nella spline
    mask = ~np.isnan(y)
    x, y = x[mask], y[mask]

    # Genera più punti per la spline
    x_smooth = np.linspace(x.min(), x.max(), num_points)
    spline = make_interp_spline(x, y, k=3)  # Cubic spline
    y_smooth = spline(x_smooth)
    return x_smooth, y_smooth

plt.figure(figsize=(20, 15))

# Plot Generator Loss - Train
plt.subplot(3, 3, 1)
for res in train_data['resolution'].unique():
    subset = train_data[train_data['resolution'] == res]
    x_smooth, y_smooth = smooth_spline(subset['Unnamed: 0'], subset['g_losses_train'])
    plt.plot(x_smooth, y_smooth, label=f'Resolution {res}')
plt.title('Generator Loss - Train')
plt.xlabel('Iteration')
plt.ylabel('Generator Loss')
plt.legend()

# Plot Discriminator Loss - Train
plt.subplot(3, 3, 2)
for res in train_data['resolution'].unique():
    subset = train_data[train_data['resolution'] == res]
    x_smooth, y_smooth = smooth_spline(subset['Unnamed: 0'], subset['d_losses_train'])
    plt.plot(x_smooth, y_smooth, label=f'Resolution {res}')
plt.title('Discriminator Loss - Train')
plt.xlabel('Iteration')
plt.ylabel('Discriminator Loss')
plt.legend()

# Plot Discriminator Accuracy - Train
plt.subplot(3, 3, 3)
for res in train_data['resolution'].unique():
    subset = train_data[train_data['resolution'] == res]
    x_smooth, y_smooth = smooth_spline(subset['Unnamed: 0'], subset['d_accuracies_train'])
    plt.plot(x_smooth, y_smooth, label=f'Resolution {res}')
plt.title('Discriminator Accuracy - Train')
plt.xlabel('Iteration')
plt.ylabel('Discriminator Accuracy')
plt.legend()

# Plot Generator Loss - Test
plt.subplot(3, 3, 4)
for res in test_data['resolution'].unique():
    subset = test_data[test_data['resolution'] == res]
    x_smooth, y_smooth = smooth_spline(subset['Unnamed: 0'], subset['g_losses_test'])
    plt.plot(x_smooth, y_smooth, label=f'Resolution {res}')
plt.title('Generator Loss - Test')
plt.xlabel('Iteration')
plt.ylabel('Generator Loss')
plt.legend()

# Plot Discriminator Loss - Test
plt.subplot(3, 3, 5)
for res in test_data['resolution'].unique():
    subset = test_data[test_data['resolution'] == res]
    x_smooth, y_smooth = smooth_spline(subset['Unnamed: 0'], subset['d_losses_test'])
    plt.plot(x_smooth, y_smooth, label=f'Resolution {res}')
plt.title('Discriminator Loss - Test')
plt.xlabel('Iteration')
plt.ylabel('Discriminator Loss')
plt.legend()

# Plot Discriminator Accuracy - Test
plt.subplot(3, 3, 6)
for res in test_data['resolution'].unique():
    subset = test_data[test_data['resolution'] == res]
    x_smooth, y_smooth = smooth_spline(subset['Unnamed: 0'], subset['d_accuracies_test'])
    plt.plot(x_smooth, y_smooth, label=f'Resolution {res}')
plt.title('Discriminator Accuracy - Test')
plt.xlabel('Iteration')
plt.ylabel('Discriminator Accuracy')
plt.legend()


# Plot Inception Score
plt.subplot(3, 3, 7)
for res in metrics_data['resolution'].unique():
    subset = metrics_data[metrics_data['resolution'] == res]
    rolling_mean = calculate_rolling_mean(subset['IS_score'])
    plt.plot(subset['Unnamed: 0'], rolling_mean, label=f'Resolution {res}')
plt.title('Inception Score')
plt.xlabel('Iteration')
plt.ylabel('Inception Score')
plt.legend()

# Plot Inception Score STD
plt.subplot(3, 3, 8)
for res in metrics_data['resolution'].unique():
    subset = metrics_data[metrics_data['resolution'] == res]
    rolling_mean = calculate_rolling_mean(subset['IS_std_score'])
    plt.plot(subset['Unnamed: 0'], rolling_mean, label=f'Resolution {res}')
plt.title('Inception Score STD')
plt.xlabel('Iteration')
plt.ylabel('Inception Score STD')
plt.legend()

# Plot MS-SSIM Scores
plt.subplot(3, 3, 9)
for res in metrics_data['resolution'].unique():
    subset = metrics_data[metrics_data['resolution'] == res]
    rolling_mean = calculate_rolling_mean(subset['ms_ssim_scores'])
    plt.plot(subset['Unnamed: 0'], rolling_mean, label=f'Resolution {res}')
plt.title('MS-SSIM Scores')
plt.xlabel('Iteration')
plt.ylabel('MS-SSIM Scores')
plt.legend()

plt.tight_layout()
plt.show()
